In [1]:
from icalendar import Calendar, Event

In [34]:
jobs = {}
i = 0

In [55]:
new_string = print('{}'.format('hello'))

SyntaxError: invalid syntax (<ipython-input-55-3f9e0731d58a>, line 1)

In [50]:
def raw(text):
    new_string=
    
    return new_string

In [51]:
def parseKeys(component):
    
      
    if component.has_key('DTSTART'):
            DTSTART = component.get('DTSTART').to_ical()
    else:
        DTSTART = None

    if component.has_key('DTEND'):
        DTEND = component.get('DTEND').to_ical()
    else:
        DTEND = None

    if component.has_key('RRULE'):
        RRULE = component.get('RRULE').to_ical()
    else:
        RRULE = None

    if component.has_key('DTSTAMP'):
        DTSTAMP = component.get('DTSTAMP').to_ical()
    else:
        DTSTAMP = None

    if component.has_key('UID'):
        UID = component.get('UID').to_ical()
    else:
        UID = None

    if component.has_key('CREATED'):
        CREATED = component.get('CREATED').to_ical()
    else:
        CREATED = None

    if component.has_key('DESCRIPTION'):
        DESCRIPTION = raw(component.get('DESCRIPTION').to_ical())
    else:
        DESCRIPTION = None

    if component.has_key('LAST-MODIFIED'):
        LASTMODIFIED = component.get('LAST-MODIFIED').to_ical()
    else:
        LASTMODIFIED = None

    if component.has_key('LOCATION'):
        LOCATION = component.get('LOCATION').to_ical()
    else:
        LOCATION = None

    if component.has_key('SEQUENCE'):
        SEQUENCE = component.get('SEQUENCE').to_ical()
    else:
        SEQUENCE = None

    if component.has_key('SUMMARY'):
        SUMMARY = raw(component.get('SUMMARY').to_ical())
    else:
        SUMMARY = None

    if component.has_key('TRANSP'):
        TRANSP = component.get('TRANSP').to_ical()
    else:
        TRANSP = None


    return [DTSTART,
            DTEND,
            RRULE,
            DTSTAMP,
            UID,
            CREATED,
            DESCRIPTION,
            LASTMODIFIED,
            LOCATION,
            SEQUENCE,
            SUMMARY,
            TRANSP]

In [52]:
with open('vectormovingsd@gmail.com.ics','rb') as fp:
    gcal = Calendar.from_ical(fp.read())
    for component in gcal.walk():
    
        if component.name == 'VEVENT':
         
            jobs[i] = parseKeys(component)

            i += 1
    
g.close()

In [25]:
import pandas as pd

In [53]:
dfJobs = pd.DataFrame.from_dict(jobs, orient='index')
dfJobs.columns = ['DTSTART',
                  'DTEND',
                  'RRULE',
                  'DTSTAMP',
                  'UID',
                  'CREATED',
                  'DESCRIPTION',
                  'LASTMODIFIED',
                  'LOCATION',
                  'SEQUENCE',
                  'SUMMARY',
                  'TRANSP']

In [54]:
dfJobs

,DTSTART,DTEND,RRULE,DTSTAMP,UID,CREATED,DESCRIPTION,LASTMODIFIED,LOCATION,SEQUENCE,SUMMARY,TRANSP
0,20130817T150000Z,20130817T160000Z,None,20161129T123158Z,ADD8F2DD-BFEC-4BF8-935F-D8AC24FCBBA8,20130816T000829Z,7 hours =$975\n15%=$146.25\nJllano@sdsp.net,20130818T175308Z,972 broadway 91911 building bihind 970 to 2811...,0,$130/hr $65 gas John Llano 619-520-3564\; 619-...,OPAQUE
1,20130818T220000Z,20130818T230000Z,None,20161129T123158Z,5BB18929-7974-4C88-9C53-95588D841D7D,20130818T173550Z,4:15 $306\n15%- $46.00,20130819T202706Z,7864 avenida navidad San Diego 92122 #195 to 8...,0,$65 $30 kar khe 832-466-5649 858-663-5613,OPAQUE
2,20130818T203000Z,20130818T213000Z,None,20161129T123158Z,97vcl68nn3potk63j78ds7vs54@google.com,20130819T203103Z,,20130819T203103Z,,0,Send 146.25+46= 192.25,OPAQUE
3,20130819T190000Z,20130819T200000Z,None,20161129T123158Z,B7461062-4494-4A2D-8E78-5538A8E980DD,20130815T205130Z,Ao.dawson88@gmail.com\nPark on beech st \n4 ho...,20130826T003509Z,1455 kertner blvd apt526 92101 to 801 ash st a...,1,$65 $30 Alex Dawson 928-301-3402,OPAQUE
4,20130820T150000Z,20130820T160000Z,None,20161129T123158Z,uitttdtrjts4vermju9k0vtf5c@google.com,20130818T005834Z,"$15 wardrobes\n3 hours\, 3wardrobes \n15% $37.05",20130826T003928Z,707 10-th ave san diego ca 92101 to storage se...,2,Andrew Garry 860-705-4143 $65 $20 5% off,OPAQUE
5,20130820T150000Z,20130820T160000Z,None,20161129T123158Z,2i941ljktu1raatvg1dlgvk1h0@google.com,20130819T185751Z,Mazi7474@yahoo.com\n19.5 hrs ( 5% off)\n15%$20...,20130826T004042Z,3435 capri way unit 1 oceanside ca 92056 to 21...,1,Mahsa Ziadloo 760-846-5094 760-846-5097 $140 $65,OPAQUE
6,20130820T210000Z,20130820T220000Z,None,20161129T123158Z,222F6FB4-22EB-4FA2-890D-F661709D98D2,20130819T211600Z,3 hours\n15%$33.75,20130826T004121Z,#224 11549 Windcrest lane 92128 to storage.,0,Alene Mcbride 858-568-3848 $65 $30,OPAQUE
7,20130821T170000Z,20130821T180000Z,None,20161129T123158Z,btfvh5trm4924q1thqdedlaslg@google.com,20130822T014246Z,3 hours \n$31.5,20130826T004213Z,6586 embrosia dr #5405 san diego 92124,1,Daby Davis 702-715-5801 $65 $15,OPAQUE
8,20130823T140000Z,20130823T150000Z,None,20161129T123158Z,0g22cflo9jscgjlkedoeca7jd4@google.com,20130823T014730Z,mosb_33@hotmail.com\n3 hours\n$33.75,20130826T004250Z,6699 beadnell apt 241 san diego 92117 to 910 d...,0,Nouf Barasayn 858-717-3392 $65 $30,OPAQUE
9,20130823T171200Z,20130823T181200Z,None,20161129T123158Z,5FF5E7A1-4FD4-4B93-8107-C2033BC6D12E,20130823T165258Z,4 hr\n$44.25,20130826T004418Z,Carlos Mantoya 832-712-4685 $65 $35,0,Carlos Mantoya 832-712-4685 $65 $35,OPAQUE


In [49]:
jobs[3]

['20130819T190000Z',
 '20130819T200000Z',
 None,
 '20161129T123158Z',
 'B7461062-4494-4A2D-8E78-5538A8E980DD',
 '20130815T205130Z',
 'Ao.dawson88@gmail.com\\nPark on beech st \\n4 hours.\\n$44.25',
 '20130826T003509Z',
 '1455 kertner blvd apt526 92101 to 801 ash st apt204 92101',
 '1',
 '$65 $30 Alex Dawson 928-301-3402 ',
 'OPAQUE']